In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


22/07/04 14:03:16 WARN Utils: Your hostname, DESKTOP-KMM8QC4 resolves to a loopback address: 127.0.1.1; using 172.24.131.74 instead (on interface eth0)
22/07/04 14:03:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/04 14:03:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/04 14:03:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
# !wget --no-check-certificate https://nyc-tlc.s3.amazoneaws.com/trip+data/fhvhv_tripdata_2021-01.csv
!wget https://s3.amazonaws.com/nyc-tlc/csv_backup/fhvhv_tripdata_2021-01.csv

--2022-07-04 14:08:40--  https://s3.amazonaws.com/nyc-tlc/csv_backup/fhvhv_tripdata_2021-01.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.142.200
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.142.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  1.85MB/s    in 12m 17s 

2022-07-04 14:20:58 (997 KB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [8]:
!ls fhvhv_tripdata_2021-01.csv

fhvhv_tripdata_2021-01.csv


In [9]:
!du -sh  fhvhv_tripdata_2021-01.csv

718M	fhvhv_tripdata_2021-01.csv


In [10]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [11]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [12]:
df.head()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None)

In [15]:
# To display the spark data types for fields
df.schema
# Here pickup_datetime & dropoff_datetime fields were string type, needs to be converted into datetime format.

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [16]:
!head -101 fhvhv_tripdata_2021-01.csv > head.csv

In [17]:
!head head.csv

In [19]:
import pandas as pd


In [20]:
pandas_df = pd.read_csv('head.csv')

In [22]:
pandas_df.dtypes


hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [23]:
spark.createDataFrame(pandas_df).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [25]:
spark.createDataFrame(pandas_df).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [26]:
### PULocationID and DOLocationID were stored as LongType that consumes 8bytes in memory, 
### this can be modified to int type(4 bytes) for better mem. utilization

In [38]:
## It is conveient to descibe custom schema types according to the operations going to be performed on this data set
## Spark is implemented in scala lang. the types in the schema should be defined as per in Scala lang definition not in python specific


In [39]:
from pyspark.sql.types import StringType, TimestampType, IntegerType, StructField,StructType


In [40]:
schema = StructType ([
    StructField('hvfhs_license_num',StringType(),True),
    StructField('dispatching_base_num',StringType(),True),
    StructField('pickup_datetime',TimestampType(),True),
    StructField('dropoff_datetime',TimestampType(),True),
    StructField('PULocationID',IntegerType(),True),
    StructField('DOLocationID',IntegerType(),True),
    StructField('SR_Flag',StringType(),True)]
)

In [41]:
## Apply schema to spark data frame


In [43]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [45]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

22/07/04 15:39:57 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2798692 ms exceeds timeout 120000 ms
22/07/04 15:39:57 WARN SparkContext: Killing executors is not supported by current scheduler.


In [46]:
df = df.repartition(24)

In [47]:
df.write.parquet('fhvhv/2021/01/')

In [48]:
!ls 'fhvhv/2021/01/'
# 24 partition files were created!

_SUCCESS
part-00000-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00001-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00002-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00003-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00004-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00005-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00006-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00007-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00008-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00009-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00010-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00011-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00012-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00013-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet
part-00014-e4c9e549-9fd3-4a97-b8b1-2cc7

In [53]:
!du -sh 'fhvhv/2021/01/part-00000-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet'

9.0M	fhvhv/2021/01/part-00000-e4c9e549-9fd3-4a97-b8b1-2cc71872198f-c000.snappy.parquet


In [54]:
df = spark.read.parquet('fhvhv/2021/01')

In [55]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [58]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').filter(df.hvfhs_license_num=='HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-04 09:56:14|2021-01-04 10:18:54|         166|         234|
|2021-01-07 01:58:26|2021-01-07 02:04:45|          35|          77|
|2021-01-06 15:13:09|2021-01-06 15:28:36|         169|          69|
|2021-01-03 16:19:52|2021-01-03 16:27:13|         170|         161|
|2021-01-01 00:33:20|2021-01-01 00:46:08|         168|          42|
|2021-01-05 23:33:36|2021-01-05 23:47:15|          35|         222|
|2021-01-06 08:53:18|2021-01-06 09:06:09|          74|         151|
|2021-01-06 09:16:53|2021-01-06 10:18:41|         168|         132|
|2021-01-02 10:51:13|2021-01-02 10:58:34|          47|          18|
|2021-01-02 21:54:41|2021-01-02 22:09:47|         239|         141|
|2021-01-05 12:07:31|2021-01-05 12:17:42|         168|          42|
|2021-01-01 19:49:13|2021-01-01 20:07:12|       

In [60]:
# Spark Operations
# 1. Transformation - lazy execitopm. select, filter, map, joins, group by
# 2. Actions - eager execution(executed immediately) . E.g show, count, take, head, write 

In [62]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').filter(df.hvfhs_license_num=='HV0003').head(10)

[Row(pickup_datetime=datetime.datetime(2021, 1, 4, 9, 56, 14), dropoff_datetime=datetime.datetime(2021, 1, 4, 10, 18, 54), PULocationID=166, DOLocationID=234),
 Row(pickup_datetime=datetime.datetime(2021, 1, 7, 1, 58, 26), dropoff_datetime=datetime.datetime(2021, 1, 7, 2, 4, 45), PULocationID=35, DOLocationID=77),
 Row(pickup_datetime=datetime.datetime(2021, 1, 6, 15, 13, 9), dropoff_datetime=datetime.datetime(2021, 1, 6, 15, 28, 36), PULocationID=169, DOLocationID=69),
 Row(pickup_datetime=datetime.datetime(2021, 1, 3, 16, 19, 52), dropoff_datetime=datetime.datetime(2021, 1, 3, 16, 27, 13), PULocationID=170, DOLocationID=161),
 Row(pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 20), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 46, 8), PULocationID=168, DOLocationID=42),
 Row(pickup_datetime=datetime.datetime(2021, 1, 5, 23, 33, 36), dropoff_datetime=datetime.datetime(2021, 1, 5, 23, 47, 15), PULocationID=35, DOLocationID=222),
 Row(pickup_datetime=datetime.datetime(2021, 1,

In [63]:
# import spark sql functions
from pyspark.sql import functions as F

In [67]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()



+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-04|  2021-01-04|         166|         234|
| 2021-01-07|  2021-01-07|          35|          77|
| 2021-01-04|  2021-01-04|         232|         161|
| 2021-01-04|  2021-01-04|         242|           3|
| 2021-01-05|  2021-01-05|         130|         260|
| 2021-01-01|  2021-01-01|          41|         133|
| 2021-01-06|  2021-01-06|         169|          69|
| 2021-01-03|  2021-01-03|         170|         161|
| 2021-01-01|  2021-01-01|         168|          42|
| 2021-01-05|  2021-01-05|          35|         222|
| 2021-01-06|  2021-01-06|          74|         151|
| 2021-01-02|  2021-01-02|          76|          76|
| 2021-01-06|  2021-01-06|         168|         132|
| 2021-01-03|  2021-01-03|         233|          95|
| 2021-01-01|  2021-01-01|          68|         265|
| 2021-01-02|  2021-01-02|          47|       

In [69]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [70]:
crazy_stuff('B02885')

'e/b45'

In [74]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=StringType())

In [77]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id','pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b30| 2021-01-04|  2021-01-04|         166|         234|
|  e/b42| 2021-01-07|  2021-01-07|          35|          77|
|  s/af0| 2021-01-04|  2021-01-04|         232|         161|
|  e/9ce| 2021-01-04|  2021-01-04|         242|           3|
|  e/9ce| 2021-01-05|  2021-01-05|         130|         260|
|  e/9ce| 2021-01-01|  2021-01-01|          41|         133|
|  e/b3f| 2021-01-06|  2021-01-06|         169|          69|
|  e/b42| 2021-01-03|  2021-01-03|         170|         161|
|  e/b3e| 2021-01-01|  2021-01-01|         168|          42|
|  s/acd| 2021-01-05|  2021-01-05|          35|         222|
|  s/acd| 2021-01-06|  2021-01-06|          74|         151|
|  e/9ce| 2021-01-02|  2021-01-02|          76|          76|
|  s/b44| 2021-01-06|  2021-01-06|         168|         132|
|  e/9ce| 2021-01-03|  2

Traceback (most recent call last):
  File "/home/thamizh/apps/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/home/thamizh/apps/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/home/thamizh/apps/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 642, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/home/thamizh/apps/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError
